In [2]:
from ccnet_spark.pipe_line import Pipeline, Config,PipelineStep
import time
from pyspark.sql import SparkSession
import sys
spark = (
    SparkSession.builder.appName("CCNETSpark_debug")
    # .master("local[*]")
    .config("spark.executor.memory", "100g")
    .config("spark.driver.memory", "100g")
    .config("spark.driver.maxResultSize", "100g")
    .config("spark.sql.execution.arrow.pyspark.enabled", "true")
    # .config("spark.executor.extraJavaOptions", "-XX:+PrintGCDetails -XX:+PrintGCDateStamps -XX:+PrintGCTimeStamps")
    # .config("spark.driver.extraJavaOptions", "-XX:+PrintGCDetails -XX:+PrintGCDateStamps -XX:+PrintGCTimeStamps")
    .getOrCreate()
)
config = Config(
        isSample=True,
        n_segments=1,
        sampleRate=0.01,
        cache_dir="../../cached_data/",
        output_dir="../../cached_data/",
        fasttext_model_path="../../cc_net/bin/lid.bin",
        lm_dir="../../cc_net/data/lm_sp",
        cutoff_csv_path="../../cc_net/cc_net/data/cutoff.csv",
        dump="2019-18",
    )
pipeline = Pipeline(config,spark)
df = pipeline.load_data()

Downloaded https://data.commoncrawl.org/crawl-data/CC-MAIN-2019-18/wet.paths.gz


2024-04-12 11:43 INFO 2046623:ccnet_spark.pipe_preprocess - Opening ../../cached_data/commoncrawl_data/2019-18/CC-MAIN-20190418101243-20190418122248-00011.warc.wet.gz with mode 'rt'


Downloaded ../../cached_data/commoncrawl_data/2019-18/CC-MAIN-20190418101243-20190418122248-00011.warc.wet.gz


2024-04-12 11:43 INFO 2046623:root - Created DataFrame with 42009 documents
2024-04-12 11:43 INFO 2046623:root - load segment 0, 420 docs, with sampleRate:1.0%,min_len:300,with dump:2019-18
2024-04-12 11:43 INFO 2046623:root - load segment 0, with sampleRate:1.0%,min_len:300,with date:2019-18


In [3]:
print(df.columns)
df.select("url","length","nlines","raw_content","cc_segment").show(5)

['url', 'date_download', 'digest', 'length', 'nlines', 'source_domain', 'title', 'raw_content', 'cc_segment']
+--------------------+------+------+----------------------+--------------------+
|                 url|length|nlines|           raw_content|          cc_segment|
+--------------------+------+------+----------------------+--------------------+
|https://www.plcca...|  8763|   355|  Toggle menu\nORDE...|CC-MAIN-201904181...|
|https://www.malar...|  6916|    78|  Skip to content\n...|CC-MAIN-201904181...|
|http://www.889906...| 92403|    63|首 页\nwww.889906.c...|CC-MAIN-201904181...|
|https://babydecor...|  8187|   411|  Koszyk: (pusty)\n...|CC-MAIN-201904181...|
|http://www.unmiko...|  8324|   104|  This Site: UNMIK ...|CC-MAIN-201904181...|
+--------------------+------+------+----------------------+--------------------+
only showing top 5 rows



In [5]:
pipeline.run_step(PipelineStep.REAL_LEN)
print(pipeline.df.columns)
pipeline.df.select("length").show(5)

['url', 'date_download', 'digest', 'length', 'nlines', 'source_domain', 'title', 'raw_content', 'cc_segment']
+------+
|length|
+------+
|  8514|
|  6663|
| 40038|
|  7803|
|  8255|
+------+
only showing top 5 rows



In [8]:
pipeline.run_step(PipelineStep.HASH)
print(pipeline.df.columns)
pipeline.df.select("length", 'hash_value').show(5)

['url', 'date_download', 'digest', 'length', 'nlines', 'source_domain', 'title', 'raw_content', 'cc_segment', 'exploded_content', 'hash_value']
+------+--------------------+
|length|          hash_value|
+------+--------------------+
|  8514|[3E 7B C7 5D 29 5...|
|  8514|[F9 73 EF EF 40 5...|
|  8514|[7F 58 0C E2 E9 5...|
|  8514|[B6 58 9F C6 AB 0...|
|  8514|[35 59 D7 AC CF 0...|
+------+--------------------+
only showing top 5 rows



In [14]:
from pyspark.sql import functions as F
pipeline.df.withColumn("lengtha", F.length(pipeline.df["hash_value"])).select('lengtha').show()

+-------+
|lengtha|
+-------+
|      8|
|      8|
|      8|
|      8|
|      8|
|      8|
|      8|
|      8|
|      8|
|      8|
|      8|
|      8|
|      8|
|      8|
|      8|
|      8|
|      8|
|      8|
|      8|
|      8|
+-------+
only showing top 20 rows

